In [2]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('/content/gdrive/My Drive/nn5/Sentiment (1).csv')

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

# Keeping only the necessary columns
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-4-9a7bba3b5a33>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-4-9a7bba3b5a33>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets
    max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 60s - loss: 0.8220 - accuracy: 0.6450 - 60s/epoch - 206ms/step
144/144 - 2s - loss: 0.7569 - accuracy: 0.6765 - 2s/epoch - 11ms/step
0.7568994760513306
0.6764962673187256


In [6]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


In [7]:
#1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [8]:
model.save('sentimentAnalysis.h5') #Saving the model

In [9]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [10]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [11]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 275ms/epoch - 275ms/step
[0.62322116 0.13747928 0.23929957]
Neutral


In [12]:
#2. Apply GridSearchCV on the source code provided in the class

In [13]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-13-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 116s - loss: 0.8244 - accuracy: 0.6466 - 116s/epoch - 156ms/step
186/186 - 2s - loss: 0.7384 - accuracy: 0.6832 - 2s/epoch - 12ms/step


744/744 - 109s - loss: 0.8253 - accuracy: 0.6485 - 109s/epoch - 147ms/step
186/186 - 2s - loss: 0.7593 - accuracy: 0.6837 - 2s/epoch - 12ms/step


744/744 - 113s - loss: 0.8252 - accuracy: 0.6466 - 113s/epoch - 152ms/step
186/186 - 3s - loss: 0.7568 - accuracy: 0.6794 - 3s/epoch - 16ms/step


744/744 - 113s - loss: 0.8264 - accuracy: 0.6484 - 113s/epoch - 152ms/step
186/186 - 2s - loss: 0.7524 - accuracy: 0.6765 - 2s/epoch - 12ms/step


...

Epoch 1/2
186/186 - 36s - loss: 0.8426 - accuracy: 0.6351 - 36s/epoch - 196ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6914 - accuracy: 0.7030 - 26s/epoch - 137ms/step
47/47 - 1s - loss: 0.7548 - accuracy: 0.6810 - 752ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8470 - accuracy: 0.6347 - 36s/epoch - 193ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6880 - accuracy: 0.7058 - 25s/epoch - 137ms/step
47/47 - 1s - loss: 0.7894 - accuracy: 0.6692 - 742ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8384 - accuracy: 0.6398 - 36s/epoch - 192ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6785 - accuracy: 0.7104 - 26s/epoch - 140ms/step
47/47 - 1s - loss: 0.7335 - accuracy: 0.6760 - 751ms/epoch - 16ms/step


Epoch 1/2
186/186 - 37s - loss: 0.8420 - accuracy: 0.6374 - 37s/epoch - 198ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6738 - accuracy: 0.7158 - 25s/epoch - 135ms/step
47/47 - 1s - loss: 0.7929 - accuracy: 0.6701 - 770ms/epoch - 16ms/step


Epoch 1/2
930/930 - 134s - loss: 0.8084 - accuracy: 0.6502 - 134s/epoch - 144ms/step
Epoch 2/2
930/930 - 121s - loss: 0.6700 - accuracy: 0.7146 - 121s/epoch - 130ms/step
Best: 0.680190 using {'batch_size': 10, 'epochs': 2}
